In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

!pwd

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content


In [3]:
!pip install wandb -qqq
%cd 'path/to/scripts/here'

import os
import numpy as np
import pandas as pd
import random
import wandb

import torch
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation, ToPILImage

from networks import *
from datasets import *
from utils import *

/content/gdrive/My Drive/IC_PHD/GTA/ACSE20-21/ACSE4


In [4]:
    # Set test transforms for normalisation 
    test_transform = Compose([
        Normalize(mean=[0.5064], std=[0.2493])
    ])
    

In [5]:
# Set training static parameters and hyperparameters
parameters = dict(
    data_name="xray-data",                                                     # will search folder ./{data_name}/train and ./{data_name}/test
    target_code={"covid":0, "lung_opacity":1, "pneumonia":2, "normal":3},      # each key should be a subfolder with the training data ./{data_name}/train/{key}
    input_height=299,
    input_width=299,
    input_channels=1,
    
    test_batch_size=64,
    
    model_name="ResNet18_FeatureExtract",                                      # will search model in networks.py, must be a subclass of nn.Module and take as input param "num_classes" referring to the number of classes for classification
    dataset="XRAY3C_TensorDataset",                                            # will search dataset in datasets.py, must be a subclass of torch.utils.data.Dataset, inputs must be img_paths, targets and transform, output of __getitem__ must be an image sample (C, H, W) and its target
    
    test_transform=test_transform,
    device=set_device("cuda"),
    )

Cuda installed! Running on GPU Tesla V100-SXM2-16GB!


In [6]:
# set up offline wandb config
config = setup_config_offline(parameters)

# Set seed
set_seed(42)

True

In [7]:
# Set up evaluation with parameters, and load trained model
path_to_trained_model = os.path.join(".", "wandb", "run-20210421_144638-29js5gok", "files", "young-plasma-45_epoch15.pth")
model, test_ds  = set_up_test(config, path_to_model=path_to_trained_model, test_transform=parameters["test_transform"])

# Get Predictions
print("\n Predicting...")
img_paths, y_preds = predict(model=model, dataset=test_ds, config=config)

# Save predictions to csv file
print("\n Saving prediction file...")
img_names = [pth.split("/")[-1].split(".")[0] for pth in img_paths]
save_preds_csv(img_names, y_preds, filename="prediction.csv")
print("\n DONE!")

Reading image paths in ./xray-data/test
Loading model:  ./wandb/run-20210421_144638-29js5gok/files/young-plasma-45_epoch15.pth
 data_name                                     :                     xray-data
 target_code                                   :               {'covid': 0, 'lung_opacity': 1, 'pneumonia': 2, 'normal': 3}
 input_height                                  :                           299
 input_width                                   :                           299
 input_channels                                :                             1
 test_batch_size                               :                            64
 model_name                                    :               ResNet18_FeatureExtract
 dataset                                       :               XRAY3C_TensorDataset
 test_transform                                :               Compose(
    Normalize(mean=[0.5064], std=[0.2493])
)
 device                                        :                  

100% (950 of 950) |######################| Elapsed Time: 0:00:04 Time:  0:00:04



 Saving prediction file...

 DONE!


In [12]:
# ! NOTE STUDENTS WON'T HAVE ACCESS TO THE test_key.csv FILE
key_filepath = os.path.join(".", "test_key.csv")
pred_filepath = os.path.join(".", "prediction.csv")
acc, keydf, testdf = get_pred_acc(key_filepath, pred_filepath)    

cat = pd.merge(keydf, testdf, on=['name'], how='inner')
print(cat)
print("\n\n ACCURACY: ", acc)


         name  target_x  target_y
0      test_0         1         3
1      test_1         3         3
2      test_2         1         3
3      test_3         3         3
4      test_4         3         3
..        ...       ...       ...
945  test_945         3         3
946  test_946         1         3
947  test_947         3         3
948  test_948         1         3
949  test_949         3         3

[950 rows x 3 columns]


 ACCURACY:  0.8621052631578947
